# Idea 4: Samples fitering - online

## Περιγραφή του αλγορίθμου
- Γίνεται εκπαίδευση με grokfast - EMA. Όταν **appl_sampl_filter** is False έχω μόνο αυτό, ενώ για True εφαρμόζω επιπλέον και την ιδέα 4 για πιο έξυπνη επιλογή δειγμάτων.
- Ο Dataloader έχει έναν custom sampler (WeightedRandomSampler) ο οποίος κάθε φορά διαλέγει ένα δείγμα με βάση κάποιο βάρος/πιθανότητα.
- Στην αρχή τα βάρη είναι όλα ίδια (ομοιόμορφη κατανομή) οπότε ο Dataloader λειτουργεί όπως συνήθως διαλέγοντας τυχαία ένα sample.
- Σε κάθε επανάληψη φτιάχνεται ένα ranking των δειγμάτων (με βάση του πόσο high frequency περιέχει το καθένα) το οποίο χρησιμοποιείται για να αποφασιστεί τι βάρος/πιθανότητα θα δοθεί σε κάθε δείγμα να επιλεγεί για εκπαίδευση. Το διάνυσμα βαρών/πιθανοτήτων ανανεώνεται κάθε **sampling_distr_upd_freq** επαναλήψεις.
- Στην κατασκευή του διανύσματος βαρών από την συνολική πιθανότητα 1 δίνουμε στα **top_k** δείγματα συνολικά **top_k_sampling_prob** (και στα υπόλοιπα length(dataset) - **top_k** δείγματα δίνουμε συνολικά το υπόλοιπο 1 - **top_k_sampling_prob**).
- Με **high_freq_better** is True ακολουθούμε την αρχική μας υπόθεση ότι τα δείγματα με high frequency είναι αυτά που θα πρέπει να ταΐσουμε το δίκτυο περισσότερο για να μάθει γρηγορότερα, για False γίνεται το αντίθετο.

## Οδηγίες χρήσης για τρέξιμο
Πήγαινε στον τίτλο **Execute training (by running main funciton)**. Πήγαινε στο parser.parse_args και όρισε τις τιμές που θες να δοκιμάσεις για grid search. Οι υπερπαράμετροι που σχετίζονται με την ιδέα 4 online είναι:

- **top_k**
- **top_k_sampling_prob**
- **high_freq_better**
- **sampling_distr_upd_freq**: Μάλλον είναι οκ στο 1 γιατί ακόμα και έτσι η εκπαίδευση δεν είναι αργή οπότε δεν έχω λόγο να το αυξήσω.

Αν κάποιος θέλει να τρέξει κάποιες τιμές για το grid search, έχω βάλει στον φάκελο και ένα αρχείο για να σημειώνουμε τις τιμές των υπερπαραμέτρων που δοκίμασε ο καθένας για να μην τρέχουμε όλοι τα ίδια. Βάλτε GPU P100 (νομίζω είναι ελαφρώς καλύτερη), εμένα για τα 100.000 βήματα που έχω βάλει να είναι το default ένα τρέξιμο που κάνω μόνο με grokfast (δηλαδή **appl_sampl_filter** is False) παίρνει περίπου **7 λεπτά** οπότε καλά είμαστε από χρόνο.



In [1]:
import kagglehub

# Maybe this is needed if you want to import private datasets 
# kagglehub.login()


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

# hojjatk_mnist_dataset_path = kagglehub.dataset_download("hojjatk/mnist-dataset")

# The dataset was uploaded from me but I made it public so you too can probably load it with this line
# _ = kagglehub.dataset_download("konstantinosbarkas/mnist-dataset-processed-from-local")

# print("Data source import complete.")


In [3]:
import sys, os
# Install the Grokfast library
!wget https://raw.githubusercontent.com/ironjr/grokfast/main/grokfast.py

sys.path.append("/kaggle/working")
os.makedirs('/kaggle/working/results/algo_online', exist_ok=True)


--2025-02-10 17:41:47--  https://raw.githubusercontent.com/ironjr/grokfast/main/grokfast.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1703 (1.7K) [text/plain]
Saving to: ‘grokfast.py.5’

grokfast.py.5       100%[===================>]   1.66K  --.-KB/s    in 0s      

2025-02-10 17:41:47 (29.0 MB/s) - ‘grokfast.py.5’ saved [1703/1703]



In [4]:
# import argparse
# import gzip
import math
import random
# import struct
import time
from argparse import ArgumentParser
# from collections import Counter, defaultdict, deque
from itertools import islice
# from pathlib import Path
# from typing import Dict, List, Literal, Optional

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from functorch import grad, vmap
from sklearn.model_selection import train_test_split
from torch.autograd import grad

# from torch.nn.utils.stateless import functional_call, # This is deprecated, use the next one instead
from torch.func import functional_call
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler, Dataset
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from itertools import islice
import torch.nn.functional as F

from grokfast import gradfilter_ema,gradfilter_ma


In [5]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enables detailed CUDA error messages

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
optimizer_dict = {"AdamW": torch.optim.AdamW, "Adam": torch.optim.Adam, "SGD": torch.optim.SGD}

activation_dict = {"ReLU": nn.ReLU, "Tanh": nn.Tanh, "Sigmoid": nn.Sigmoid, "GELU": nn.GELU}

loss_function_dict = {"MSE": nn.MSELoss, "CrossEntropy": nn.CrossEntropyLoss}


In [8]:
class Block(nn.Module):
    """Causal transformer block
    """

    def __init__(self, dim, num_heads):
        super().__init__()
        self.ln_1 = nn.LayerNorm(dim)
        self.ln_2 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads)
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Linear(dim * 4, dim),
        )

    def forward(self, x):
        attn_mask = torch.full(
            (len(x), len(x)), -float("Inf"), device=x.device, dtype=x.dtype
        )
        attn_mask = torch.triu(attn_mask, diagonal=1)
        attn_mask[torch.isnan(attn_mask)] = 0.0 # fixes all 'nan' on 'mps' device

        x = self.ln_1(x)
        a, _ = self.attn(x, x, x, attn_mask=attn_mask, need_weights=False)
        x = x + a
        m = self.mlp(self.ln_2(x))
        x = x + m
        return x


In [9]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    """Causal Transformer Decoder"""

    def __init__(self, dim=128, num_layers=2, num_heads=4, num_tokens=97, seq_len=5):
        super().__init__()
        self.token_embeddings = nn.Embedding(num_tokens, dim)
        self.position_embeddings = nn.Embedding(seq_len, dim)
        self.layers = nn.ModuleList([Block(dim, num_heads) for _ in range(num_layers)])

        self.ln_f = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, num_tokens, bias=False)

    def forward(self, x):
        # Ensure input is a LongTensor for embeddings
        x = x.long()
        
        h = self.token_embeddings(x)  # Token embeddings
        batch_size, seq_len = x.shape  # Get batch size and sequence length
        
        # Ensure positions are correctly shaped
        positions = torch.arange(seq_len, device=x.device).expand(batch_size, seq_len)
        print(f"positions.min() = {positions.min()}, positions.max() = {positions.max()}")  # Debug print

        # Ensure positions are within bounds (e.g., between 0 and num_embeddings-1)
        positions = torch.clamp(positions, 0, self.position_embeddings.num_embeddings - 1)
        
        # Convert to long type (required for embedding layer)
        positions = positions.long()
        # Get position embeddings
        position_embeddings = self.position_embeddings(positions)

        # Debugging prints
        print(f"x shape: {x.shape}, device: {x.device}")
        print(f"h shape: {h.shape}, device: {h.device}")
        print(f"positions shape: {positions.shape}, device: {positions.device}")
        print(f"position_embeddings shape: {position_embeddings.shape}, device: {position_embeddings.device}")

        # Ensure embeddings are correctly added
        if h.shape != position_embeddings.shape:
            raise ValueError(f"Shape mismatch: h {h.shape} != position_embeddings {position_embeddings.shape}")

        h = h + position_embeddings  # Combine token and position embeddings
        
        # Pass through Transformer blocks
        for layer in self.layers:
            h = layer(h)

        h = self.ln_f(h)
        logits = self.head(h)
        return logits


In [10]:
def multiplication_mod_p_data(p, eq_token, op_token):
    """x◦y = x/y (mod p) for 0 ≤ x < p, 0 < y < p
    """
    x = torch.arange(p)
    y = torch.arange(1, p)
    x, y = torch.cartesian_prod(x, y).T

    eq = torch.ones_like(x) * eq_token
    op = torch.ones_like(x) * op_token
    result = x * y % p

    # "All of our experiments used a small transformer trained on datasets of
    # equations of the form a◦b = c, where each of “a”, “◦”, “b”, “=”, and “c”
    # is a seperate token"
    return torch.stack([x, op, y, eq, result])

In [11]:
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [12]:
def custom_collate_fn_2(batch):
    """Custom collate function to handle extra fields in the dataset."""
    images, labels, _, _ = zip(*batch)  # Ignore the indices and extra_fields for loss computation
    images = torch.stack(images)  # Stack images into a single tensor
    labels = torch.tensor(labels)  # Convert labels to a tensor
    return images, labels


In [21]:
# Added extra fields to keep ema_gra and history
# In this implementation I use variance metric --> I don't also store deviation metric for memory efficiency
class MyAlgo(torch.utils.data.Dataset):
    def __init__(self, dataset,targets, transform=None):
        """
        Custom dataset to extend Algorithmic with extra fields.
        """
        
        self.data = dataset.data.float()
        self.targets = targets
        self.transform = transform

        # Initialize extra fields
        self.extra_fields = [
            {
                "ema_grad": 0.0,
                "num_updates": 0,
                "variance_metric": 0.0,
            }
            for _ in range(self.data.shape[1])  # Change this from len(self.data) to self.data.shape[1]
        ]
        
    def __getitem__(self, index):
        print(f"self.data shape: {self.data.shape}")
        print(f"self.targets shape: {self.targets.shape}")
        print(f"Index: {index}")
        
        # Extracting the indices correctly
        if isinstance(index, tuple):
            row_idx = index[0]
            col_idx = index[1]
        else:
            row_idx = index
            col_idx = None
        
        # Ensure valid indexing
        if col_idx is None:
            x, target = self.data[row_idx], self.targets[row_idx]
        else:
            x, target = self.data[:, col_idx], self.targets[col_idx]
        
        # Normalize image data if needed
        x = torch.tensor(x).float() / 255.0  # Ensure x is a tensor and normalize it to [0, 1]
        
        # Apply any other transformations if they exist
        if self.transform:
            x = self.transform(x)
        
        # Handle slicing case separately
        if isinstance(index, slice):
            # If index is a slice, return corresponding `extra_field` entries
            extra_fields_subset = self.extra_fields[index]
            if isinstance(extra_fields_subset, list):
                # Ensure we are accessing a list of dictionaries
                if isinstance(extra_fields_subset[0], dict):
                    ema_grad = [entry["ema_grad"] for entry in extra_fields_subset]
                    num_updates = [entry["num_updates"] for entry in extra_fields_subset]
                    variance_metric = [entry["variance_metric"] for entry in extra_fields_subset]
                else:
                    raise ValueError(f"Expected list of dictionaries, but got: {type(extra_fields_subset[0])}")
            else:
                raise ValueError(f"Expected extra_fields to be a list of dictionaries, but got: {type(extra_fields_subset)}")
        else:
            # Normal index access, fetch the dictionary for a specific index
            extra_field = self.extra_fields[index]
            
            if isinstance(extra_field, dict):
                ema_grad = extra_field["ema_grad"]
                num_updates = extra_field["num_updates"]
                variance_metric = extra_field["variance_metric"]
            else:
                raise ValueError(f"Expected extra_field to be a dictionary, but got: {type(extra_field)}")
        
        # Return x, the relevant extra fields, and target
        return x, ema_grad, num_updates, variance_metric, target



    def __len__(self):
        return len(self.data)
        
    def update_fields(self, indices, grad_stats, ema_alpha=0.9):
        """
        Update the extra fields for specified dataset indices.
        """

        for idx, grad in zip(indices, grad_stats):
            # Update EMA
            sample_field = self.extra_fields[idx]

            current_ema = sample_field["ema_grad"]
            updated_ema = ema_alpha * current_ema + (1 - ema_alpha) * grad
            sample_field["ema_grad"] = updated_ema

            deviation = abs(grad - current_ema)

            num_updates = sample_field["num_updates"] + 1  # Increment the update count
            
            current_avg_deviation = sample_field["variance_metric"] ** 0.5
            new_avg_deviation = ((current_avg_deviation * (num_updates - 1)) + deviation) / num_updates

            sample_field["num_updates"] = num_updates

            # Variance estimate (for higher sensitivity to fast changes)
            sample_field["variance_metric"] = new_avg_deviation**2
            
    @property
    def shape(self):
        """
        Mimics the behavior of a tensor's shape by returning the shape of the dataset.
        """
        return self.data.shape[1]
        
    def split(self, batch_size, dim=0):
        """
        Splits the dataset into batches of size `batch_size` along the specified dimension.
        Returns a list of `MyAlgo` instances (or whatever splitting method you prefer).
        """
        # Assuming self.data is a tensor of shape (num_samples, features)
        num_samples = len(self.data)
        batch_indices = torch.arange(0, num_samples, batch_size)

        # Create a list to hold the batched datasets
        batched_datasets = []

        for i in batch_indices:
            # Slice the data tensor into batches
            batch_data = self.data[i:i + batch_size]
            batch_targets = self.targets[i:i + batch_size]
            batch_extra_fields = self.extra_fields[i:i + batch_size]
            
            # Create a new MyAlgo instance for each batch
            batched_datasets.append(MyAlgo(batch_data, batch_targets, self.transform))
        
        return batched_datasets
        
    def to(self, device):
        """Move dataset to the specified device."""
        self.data = self.data.to(device)
        self.targets = self.targets.to(device)  # Fix this line, use self.targets instead of s
        return self

In [14]:
def custom_collate_fn(batch):
    x, labels, indices, extra_fields = zip(*batch)
    x = torch.stack(x)  # Stack inputs into a single tensor
    labels = torch.tensor(labels)  # Convert labels to a tensor
    return x, labels, indices, extra_fields

In [15]:
# Needed for per sample gradient computations
def select_random_subset(tensor, percentage, seed=42):
    """
    Flatten the parameter dimensions for each batch sample, select a percentage of elements,
    and return a tensor with shape [batch_size, selected_elements].

    Args:
        tensor (torch.Tensor): The gradient tensor of shape [batch_size, *parameter_dims].
        percentage (float): The percentage of elements to select.
        seed (int): Random seed for reproducibility.

    Returns:
        torch.Tensor: A tensor of shape [batch_size, selected_elements].
    """
    batch_size, *param_dims = tensor.shape  # Extract batch and parameter dimensions
    total_params = torch.prod(torch.tensor(param_dims))  # Total parameters per sample
    subset_size = int(total_params * percentage)  # 20% of parameters

    # Set seed for reproducibility
    random.seed(seed)
    indices = random.sample(range(total_params), subset_size)  # Random indices for selection

    # Flatten parameter dimensions and select elements for each batch
    flat_tensor = tensor.view(batch_size, -1)  # Flatten parameter dimensions for each sample
    selected_subset = flat_tensor[:, indices]  # Select the same random indices across the batch

    return selected_subset


In [16]:
# Needed for online sample filtering
def rank_to_sampling_weights(my_dataset, top_k, top_k_sampling_prob, high_freq_better):
    """
    Rank samples by variance_metric and assign sampling weights.

    Parameters:
    - my_dataset: MyMNIST object.
    - top_k: Fraction of top samples to assign higher sampling probability.
    - top_k_sampling_prob: Probability assigned to the top_k fraction of samples.

    Returns:
    - new_weights: List of sampling weights for each sample.
    """
    # Calculate the number of top_k samples
    num_samples = len(my_dataset)
    top_k_count = int(top_k * num_samples)

    # Sort indices by variance_metric in descending order
    sorted_indices = sorted(
        range(num_samples),
        key=lambda idx: my_dataset.dataset.extra_fields[idx]["variance_metric"],
        reverse=high_freq_better,
    )

    # Initialize new_weights with zeros
    new_weights = [0.0] * num_samples

    # Assign weights to the top_k samples
    for idx in sorted_indices[:top_k_count]:
        new_weights[idx] = top_k_sampling_prob / top_k_count

    # Assign weights to the rest of the samples
    for idx in sorted_indices[top_k_count:]:
        new_weights[idx] = (1 - top_k_sampling_prob) / (num_samples - top_k_count)

    return new_weights


## def main

In [41]:
def main(args):
    torch.manual_seed(args.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    eq_token = args.p
    op_token = args.p + 1
    
    # Create model
    model = Decoder(
        dim=128, num_layers=2, num_heads=4, num_tokens=args.p + 2, seq_len=5
    ).to(device)
    print(model)
    nparams = sum([p.numel() for p in model.parameters() if p.requires_grad])
    print(f'Total number of parameters: {nparams}')
    for name, param in model.named_parameters():
        print(name)

    data = multiplication_mod_p_data(args.p, eq_token, op_token)
    
    # Split the subset into training and validation sets
    split_idx = data.shape[1] // 2
    perm = torch.randperm(data.shape[1])
    train_idx = perm[:split_idx]
    valid_idx = perm[split_idx:]
    train_data = MyAlgo(data[:, train_idx], targets=data[:, train_idx])
    valid_data = MyAlgo(data[:, valid_idx], targets=data[:, valid_idx])

    # Create initial weights for uniform sampling
    weights = [1.0] * len(train_data)
    sampler = WeightedRandomSampler(weights, len(weights))

    # DataLoader
    train_loader = DataLoader(train_data, batch_size=args.batch_size, sampler=sampler, collate_fn=custom_collate_fn)

    data_iter = cycle(train_loader)
    
    activation_fn = activation_dict[args.activation]

    # Optimizer
    optimizer = getattr(torch.optim, args.optimizer)(
        model.parameters(),
        lr=args.lr,
        weight_decay=args.weight_decay,
        betas=(args.beta1, args.beta2),
    )

    # Scheduler
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda update: 1 if update > 10 else update / 10
    )
    
    steps_per_epoch = math.ceil(data.shape[0] / args.batch_size)

    # Start Training below
    its, train_acc, val_acc, train_loss, val_loss = [], [], [], [], []
    grads = None
    i = 0

    # For logging network weights.
    net_its, nets = [], []

    stable_steps = 0
    stable_threshold = 10
    reached_early_stop = False
    steps_to_reach_val_acc = None

    for e in tqdm(range(int(args.budget) // steps_per_epoch)):

        # randomly shuffle train data 
        train_data.data = train_data.data[:, torch.randperm(train_data.data.shape[1])]

        for data, is_train in [(train_data, True), (valid_data, False)]:

            model.train(is_train)
            total_loss = 0
            total_acc = 0

            # Unpack data
            x, ema_grad, num_updates, variance_metric, target = data  
            dl = torch.split(data, args.batch_size, dim=0)
            
            for input in dl:
                input = input.to(device)

                with torch.set_grad_enabled(is_train):
                    x, ema_grad, num_updates, variance_metric, target = input 
                    print(x.shape)
                    print(x)
                    logits = model(x)  # Model forward pass
                    logits = logits[-1].float()  # Ensure logits are in float32
                    target = input[-1].long()  # Ensure target is long (int64)

                    loss = F.cross_entropy(logits, target)  # Compute the loss
                    total_loss += loss.item() * input.shape[-1]
                    
                if is_train:
                    model.zero_grad()
                    loss.backward()

                    # Gradient filtering logic
                    trigger = i < 500 if args.two_stage else False
                    if args.filter == "none":
                        pass
                    elif args.filter == "ma":
                        grads = gradfilter_ma(model, grads=grads, window_size=args.window_size, lamb=args.lamb, trigger=trigger)
                    elif args.filter == "ema":
                        grads = gradfilter_ema(model, grads=grads, alpha=args.alpha, lamb=args.lamb)
                    else:
                        raise ValueError(f"Invalid gradient filter type `{args.filter}`")

                    optimizer.step()
                    scheduler.step()
                    i += 1
                    
                # Compute accuracy
                acc = (logits[-1].argmax(-1) == input[-1]).float().mean()
                total_acc += acc.item() * input.shape[-1]

            if is_train:
                train_acc.append(total_acc / train_data.shape[-1])
                train_loss.append(total_loss / train_data.shape[-1])
                its.append(i)
            else:
                val_acc.append(total_acc / valid_data.shape[-1])
                val_loss.append(total_loss / valid_data.shape[-1])

        # Update sampling distribution periodically
        if args.apply_sample_filter and e % args.sampling_distr_upd_freq == 0:
            new_weights = rank_to_sampling_weights(train_data, args.top_k, args.top_k_sampling_prob, args.high_freq_better)
            new_sampler = WeightedRandomSampler(new_weights, num_samples=len(train_data), replacement=True)
            train_loader = DataLoader(
                train_data,
                batch_size=args.batch_size,
                sampler=new_sampler,
                collate_fn=custom_collate_fn
            )
            del data_iter
            data_iter = cycle(train_loader)

   
              # Update sample distribution based on gradient statistics

        # per-sample gradient computations for the Decoder model
        if args.appl_sampl_filter:
            # Define a function for forward + loss computation
            def compute_loss_vmap(params, buffers, model, x, y):
                # Forward pass through the model
                logits = model(x)  # `x` is the input batch
        
                # Here, we assume the target `y` corresponds to the expected output for classification
                loss = F.cross_entropy(logits[-1], y)  # Loss computed using the output of the final layer
                return loss.mean()
        
            # Prepare model parameters and buffers
            params_and_buffers = {**dict(model.named_parameters()), **dict(model.named_buffers())}
            
            # Separate the parameters that require gradients and those that don't
            params = {k: v for k, v in params_and_buffers.items() if v.requires_grad}
            buffers = {k: v for k, v in params_and_buffers.items() if not v.requires_grad}
            
            # Create the gradient function using torch.autograd.grad or vmap
            gradient_fn = grad(compute_loss_vmap)
            
            # Initialize EMA and metric history for each sample (if necessary)
            gradient_ema = [0.0 for _ in range(len(train_subset))]  # If you're storing gradient EMA for each sample

        
        if args.appl_sampl_filter and e % args.sampling_distr_upd_freq == 0:
            # -----------------------------------------------------------------
            # Gradient Stats: Capture gradients for each sample
            # -----------------------------------------------------------------
            with torch.no_grad():
                # Capture gradients for the layers dynamically
                per_sample_grads = vmap(gradient_fn, in_dims=(None, None, None, 0, 0))(
                    params, buffers, model, x, labels
                )
            
                # Extract gradients for the target layers
                last_layer_grad = per_sample_grads["layers.1.mlp.2.weight"]  # Last Linear layer in the second block
                second_last_layer_grad = per_sample_grads["layers.0.mlp.2.weight"]  # Last Linear layer in the first block
                final_layer_grad = per_sample_grads["head.weight"]  # Final output layer
            
                # Select a subset of gradients (percentage selection as before)
                percentage_s_l = 0.2
                percentage_l = 1
                selected_last = select_random_subset(last_layer_grad, percentage_l, seed=42)
                selected_second_last = select_random_subset(second_last_layer_grad, percentage_s_l, seed=42)
                selected_final = select_random_subset(final_layer_grad, percentage_s_l, seed=42)  # Adding final layer gradient
            
                # Compute the average of selected gradients and detach
                selected_last_avg = selected_last.mean(dim=-1).detach().cpu()
                selected_second_last_avg = selected_second_last.mean(dim=-1).detach().cpu()
                selected_final_avg = selected_final.mean(dim=-1).detach().cpu()  # Average for the final layer
            
                # Combine the averages of selected gradients
                total_avg = (selected_last_avg + selected_second_last_avg + selected_final_avg) / 3

            # Update the dataset with the computed average gradients
            train_data.dataset.update_fields(train_idx, total_avg, args.ema_alpha_sampl_rank)
        
        # Early Stopping Logic
        val_acc_last = val_acc[-1] if len(val_acc) > 0 else 0
        if val_acc_last >= 0.92 and steps_to_reach_val_acc is None:
            steps_to_reach_val_acc = i

        # Check for stable performance
        if val_acc_last > 0.9:
            stable_steps += 1
        else:
            stable_steps = 0  # Reset counter if accuracy drops below 0.85

        if stable_steps >= stable_threshold and val_acc_last >= 0.9 and steps_to_reach_val_acc is not None:
            reached_early_stop = True
            print(f"Validation accuracy of 0.92 reached and remained > 0.9 for {stable_threshold} steps at step {i}")

        if reached_early_stop:
            print("Early stopping triggered.")
            break


    # Save results
    specific_result_dir = f"algo_{args.label}.pt"
    results_filename = os.path.join(results_dir, specific_result_dir)
    torch.save(
        {
            "its": its,
            "train_acc": train_acc,
            "train_loss": train_loss,
            "val_acc": val_acc,
            "val_loss": val_loss,
            "steps_to_reach": None,  # Steps to reach validation accuracy
            "model_state_dict": model.state_dict(),
        },
        results_filename,
    )

    print(f"\nTraining complete!")
    print(f"Steps to reach 0.9 validation accuracy: {steps_to_reach_val_acc}")


In [42]:
# Remove the extra arguments passed by the Jupyter Notebook kernel
sys.argv = [""]


In [43]:
# Functions for creating plot
import os
import torch
import matplotlib.pyplot as plt

def analyze_results(label, results_dir="/kaggle/working/results/algo_online"):
    """
    Loads model results, extracts accuracy/loss data, and generates plots.
    
    Args:
        label (str): Label identifier for the results file.
        results_dir (str): Directory where results are stored.
    
    Returns:
        None
    """

    # Define file paths
    filename = f"mnist_{label}.pt"
    results_filename = os.path.join(results_dir, filename)

    filename_plot_acc = f"mnist_{label}_acc.png"
    results_filename_plot_acc = os.path.join(results_dir, filename_plot_acc)

    filename_plot_loss = f"mnist_{label}_loss.png"
    results_filename_plot_loss = os.path.join(results_dir, filename_plot_loss)

    try:
        # Load results
        results = torch.load(results_filename, weights_only=True)

        # Extract data
        its = results["its"]  # Optimization steps
        train_acc = results["train_acc"]  # Training accuracy
        val_acc = results["val_acc"]  # Validation accuracy
        train_loss = results["train_loss"]  # Training loss
        val_loss = results["val_loss"]  # Validation loss
        steps_to_reach = results["steps_to_reach"]  # Steps to reach 90% validation accuracy

        print(f"Steps needed to reach 0.9 validation accuracy: {steps_to_reach}")

        # Plot Accuracy
        plt.figure()
        plt.plot(its, train_acc, label="Train Accuracy")
        plt.plot(its, val_acc, label="Validation Accuracy")

        # Find and annotate the maximum validation accuracy
        max_val_acc = max(val_acc)
        max_val_idx = val_acc.index(max_val_acc)
        plt.annotate(f"Max Val Acc: {max_val_acc:.4f}", 
                     (its[max_val_idx], max_val_acc), 
                     textcoords="offset points", 
                     xytext=(0, 10), 
                     ha='center', 
                     fontsize=10, 
                     color='red')

        plt.legend()
        plt.title(f"Accuracy - Grokfast:{args.filter} Sample Filtering:{args.appl_sampl_filter}")
        plt.xlabel("Optimization Steps")
        plt.ylabel("Accuracy")
        plt.xscale("log", base=10)
        plt.grid()
        plt.figtext(0.5, -0.1, f"Steps to reach val=0.9 = {steps_to_reach}", 
            ha="center", fontsize=10, style="italic")
       # plt.savefig(results_filename_plot_acc, dpi=150)
        plt.show()
        plt.close()

        print("Plots saved successfully.")

    except FileNotFoundError:
        print(f"Error: Results file {results_filename} not found.")
    except Exception as e:
        print(f"Error while processing {label}: {e}")

def plot_all_experiments_together(labels, results_dir="/kaggle/working/results/mnist_online", show_only_val=False):
    """
    Plots train and validation accuracy for multiple experiments in a single graph.
    Allows showing only validation accuracy if `show_only_val=True`.

    Args:
        labels (list of str): List of labels corresponding to result files.
        results_dir (str): Directory where results are stored.
        show_only_val (bool): If True, only plots validation accuracy.

    Returns:
        None
    """
    plt.figure(figsize=(10, 6))  # Set figure size

    # Generate distinct colors for each experiment
    base_colors = plt.cm.viridis(np.linspace(0, 1, len(labels)))  

    for i, label in enumerate(labels):
        results_filename = os.path.join(results_dir, f"mnist_{label}.pt")

        try:
            # Load results
            results = torch.load(results_filename, weights_only=True)
            its = results["its"]
            train_acc = results["train_acc"]
            val_acc = results["val_acc"]

            # Assign colors for train and validation curves
            val_color = base_colors[i]  # Primary color for validation
            train_color = tuple(c * 0.7 for c in base_colors[i])  # Slightly darker shade for train
            steps_to_reach = results["steps_to_reach"]
            
            # Plot validation accuracy (always shown)
            plt.plot(its, val_acc, label=f"Validation ({label})", color=val_color, linestyle="solid")
            plt.figtext(0.5, -0.1, f"Steps to reach val=0.9 = {steps_to_reach}", 
                ha="center", fontsize=10, style="italic")
        
            # Plot train accuracy if `show_only_val` is False
            if not show_only_val:
                plt.plot(its, train_acc, label=f"Train ({label})", color=train_color, linestyle="dashed")

        except FileNotFoundError:
            print(f"Warning: Results file {results_filename} not found.")
        except Exception as e:
            print(f"Error while processing {label}: {e}")

    plt.legend()
    plt.title("Train & Validation Accuracy for Multiple Experiments" if not show_only_val else "Validation Accuracy for Multiple Experiments")
    plt.xlabel("Optimization Steps")
    plt.ylabel("Accuracy")
    plt.xscale("log", base=10)
    plt.grid()
    
    # Save and show the plot
    filename = "combined_train_val_plot.png" if not show_only_val else "combined_val_plot.png"
    plot_path = os.path.join(results_dir, filename)
    #plt.savefig(plot_path, dpi=150)
    plt.show()

## Execute training (by running main function)

From now on i just train networks with different configurations every timeand then I print their results after.

Below is Benchmark:

    * no grokfast applied
    * no filtering
    * wd = 0



            

In [44]:
if __name__ == "__main__":
    # Same as used in paper of Grokfast
    parser = ArgumentParser(description="Train a model on Algorithmic Dataset without custom sampling")
    
    parser.add_argument("--label", type=str, default="")
    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument("--p", type=int, default=97)
    parser.add_argument("--budget", type=int, default=3e5)
    parser.add_argument("--train_points", type=int, default=1000)
    parser.add_argument("--optimization_steps", type=int, default=100000)
    parser.add_argument("--batch_size", type=int, default=512)
    parser.add_argument("--optimizer", type=str, default="Adam")
    parser.add_argument("--beta1", type=float, default=0.9)
    parser.add_argument("--beta2", type=float, default=0.98)
    parser.add_argument("--weight_decay", type=float, default=0)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--initialization_scale", type=float, default=8.0)
    parser.add_argument("--download_directory", type=str, default=".")
    parser.add_argument("--depth", type=int, default=3)
    parser.add_argument("--width", type=int, default=200)
    parser.add_argument("--activation", type=str, default="ReLU")

    # Grokfast
    parser.add_argument("--filter", type=str, choices=["none", "ma", "ema", "fir"], default="none")
    parser.add_argument("--alpha", type=float, default=0.99)
    parser.add_argument("--window_size", type=int, default=100)
    parser.add_argument("--lamb", type=float, default=0.1)

    # Ablation studies
    parser.add_argument("--two_stage", action='store_true')
    parser.add_argument("--save_weights", action='store_true')

    # Samples ranking
    parser.add_argument("--ema_alpha_sampl_rank", type=float, default=0.9)

    # Boolean arguements need this due to bad behavior of parser.parse_args
    def boolean_string(s):
        if s not in {"False", "True"}:
            raise ValueError("Not a valid boolean string")
        return s == "True"

    # These are the hyperparameters related to our online sampling filtering algorithm
    parser.add_argument("--appl_sampl_filter", type=boolean_string, default=True)  # If False, perform regular training
    parser.add_argument("--sampling_distr_upd_freq", type=int, default=1)  # How often to update the sampling distribution
    parser.add_argument("--top_k", type=float, default=0.1)  # Fraction of samples to select more frequently
    parser.add_argument("--top_k_sampling_prob", type=float, default=0.9)  # Probability of selecting a sample from the top-k
    parser.add_argument("--high_freq_better", type=boolean_string, default=True)  # If True, samples with higher frequency gradient content are considered better for training

    # -----------------------------------------------------------------
    # Try different hyperparameter values for your grid search here
    # -----------------------------------------------------------------
    args = parser.parse_args(
        [
            "--appl_sampl_filter", "False", # booleans as non strings in order to work
            "--sampling_distr_upd_freq", "1", # the rest as strings for some reason
            "--top_k", "0.1",
            "--top_k_sampling_prob", "0.9",
            "--high_freq_better", "True",
        ]
    )
    # -----------------------------------------------------------------
    # -----------------------------------------------------------------

    # Create arg.label for the filename of the saved results
    if not args.appl_sampl_filter:
        args.label = f"filter{args.filter}_sampling_{args.appl_sampl_filter}"
    else:
        args.label = f"high_freq_{args.high_freq_better}_top_k_{args.top_k}_top_k_prob_{args.top_k_sampling_prob}_upd_freq_{args.sampling_distr_upd_freq}"

    # Training with time recording

    # Start the timer
    start_time = time.time()

    # Call your training function
    main(args)

    # End the timer
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Convert to minutes and seconds (optional)
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training completed in {int(minutes)} minutes and {int(seconds)} seconds.")
    print(f"label:{args.label}")

Decoder(
  (token_embeddings): Embedding(99, 128)
  (position_embeddings): Embedding(5, 128)
  (layers): ModuleList(
    (0-1): 2 x Block(
      (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=128, bias=True)
      )
    )
  )
  (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=128, out_features=99, bias=False)
)
Total number of parameters: 422784
token_embeddings.weight
position_embeddings.weight
layers.0.ln_1.weight
layers.0.ln_1.bias
layers.0.ln_2.weight
layers.0.ln_2.bias
layers.0.attn.in_proj_weight
layers.0.attn.in_proj_bias
layers.0.attn.out_pro

  0%|          | 0/300000 [00:00<?, ?it/s]

self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 0
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 1
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 2
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 3
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 4
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 5
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 0
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 1
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 2
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 3
self.data shape: torch.Size([5, 4656])
self.targets shape: torch.Size([5, 4656])
Index: 4
self.data 

<ipython-input-21-7a6205a64be1>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).float() / 255.0  # Ensure x is a tensor and normalize it to [0, 1]


AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
args.label="filternone_sampling_False" analyze_results(args.label)

In [32]:
if __name__ == "__main__":
    # Same as used in paper of Grokfast
    parser = ArgumentParser(description="Train a model on MNIST without custom sampling")
    
    parser.add_argument("--label", type=str, default="")
    parser.add_argument("--seed", type=int, default=0)
    
    parser.add_argument("--train_points", type=int, default=1000)
    parser.add_argument("--optimization_steps", type=int, default=100000)
    parser.add_argument("--batch_size", type=int, default=200)
    parser.add_argument("--loss_function", type=str, default="MSE") #MSE or CrossEntropy
    parser.add_argument("--optimizer", type=str, default="AdamW")
    parser.add_argument("--weight_decay", type=float, default=2.0)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--initialization_scale", type=float, default=8.0)
    parser.add_argument("--download_directory", type=str, default=".")
    parser.add_argument("--depth", type=int, default=3)
    parser.add_argument("--width", type=int, default=200)
    parser.add_argument("--activation", type=str, default="ReLU")

    # Grokfast
    parser.add_argument("--filter", type=str, choices=["none", "ma", "ema", "fir"], default="ema")
    parser.add_argument("--alpha", type=float, default=0.8)
    parser.add_argument("--lamb", type=float, default=0.1)

    # Samples ranking
    parser.add_argument("--ema_alpha_sampl_rank", type=float, default=0.9)

    # Boolean arguements need this due to bad behavior of parser.parse_args
    def boolean_string(s):
        if s not in {"False", "True"}:
            raise ValueError("Not a valid boolean string")
        return s == "True"

    # These are the hyperparameters related to our online sampling filtering algorithm
    parser.add_argument("--appl_sampl_filter", type=boolean_string, default=True)  # If False, perform regular training
    parser.add_argument("--sampling_distr_upd_freq", type=int, default=1)  # How often to update the sampling distribution
    parser.add_argument("--top_k", type=float, default=0.1)  # Fraction of samples to select more frequently
    parser.add_argument("--top_k_sampling_prob", type=float, default=0.9)  # Probability of selecting a sample from the top-k
    parser.add_argument("--high_freq_better", type=boolean_string, default=True)  # If True, samples with higher frequency gradient content are considered better for training

    # -----------------------------------------------------------------
    # Try different hyperparameter values for your grid search here
    # -----------------------------------------------------------------
    args = parser.parse_args(
        [
            "--appl_sampl_filter", "False", # booleans as non strings in order to work
            "--sampling_distr_upd_freq", "1", # the rest as strings for some reason
            "--top_k", "0.1",
            "--top_k_sampling_prob", "0.9",
            "--high_freq_better", "True",
        ]
    )
    # -----------------------------------------------------------------
    # -----------------------------------------------------------------

    # Create arg.label for the filename of the saved results
    if not args.appl_sampl_filter:
        args.label = f"filter{args.filter}_sampling_{args.appl_sampl_filter}"
    else:
        args.label = f"high_freq_{args.high_freq_better}_top_k_{args.top_k}_top_k_prob_{args.top_k_sampling_prob}_upd_freq_{args.sampling_distr_upd_freq}"

    # Training with time recording

    # Start the timer
    start_time = time.time()

    # Call your training function
    main(args)

    # End the timer
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Convert to minutes and seconds (optional)
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training completed in {int(minutes)} minutes and {int(seconds)} seconds.")
    print(f"label:{args.label}")

AttributeError: 'Namespace' object has no attribute 'p'

In [ ]:
args.label="filterema_sampling_False"
analyze_results(args.label)

In [ ]:
# Define possible values for each parameter
param_grid = {
    "top_k": [0.2],  # Convert to float
    "top_k_sampling_prob": [0.7],  # Convert to float
    "sampling_distr_upd_freq": [10],  # Convert to int
    "high_freq_better": [True]  # Boolean parameter
}

# Generate all combinations of parameters
param_combinations = list(itertools.product(*param_grid.values()))

# Run main in a loop for each combination
for param_values in param_combinations:
    # Extract parameter values
    top_k = param_values[0]
    top_k_sampling_prob = param_values[1]
    sampling_distr_upd_freq = param_values[2]
    high_freq_better = param_values[3]  # Boolean value
    
    # Ensure boolean values are correctly formatted as strings for argparse
    high_freq_better_str = "True" if high_freq_better else "False"

    # Create args dynamically
    args_list = [
        "--appl_sampl_filter" , "True",
        "--top_k", str(top_k),
        "--top_k_sampling_prob", str(top_k_sampling_prob),
        "--sampling_distr_upd_freq", str(sampling_distr_upd_freq),
        "--high_freq_better", high_freq_better_str,
        "--label", f"{top_k}_{top_k_sampling_prob}_{sampling_distr_upd_freq}_{high_freq_better}"
    ]

    # Debug print statement
    print(f"\nRunning with parameters: {args_list}")

    # Parse the arguments dynamically
    args = parser.parse_args(args_list)

    # Call main() with the updated args
    main(args)

In [ ]:
#/kaggle/working/results/mnist_online/mnist_high_freq_True_top_k_0.2_top_k_prob_0.9_upd_freq_1.pt
args.label="high_freq_True_top_k_0.2_top_k_prob_0.9_upd_freq_1"
analyze_results(args.label)

In [ ]:
    args_list = [
        "--appl_sampl_filter" , "False",
    ]

    # Parse the arguments dynamically
    args = parser.parse_args(args_list)

    # Call main() with the updated args
    main(args)

In [ ]:
analyze_results(args.label)

In [ ]:
if __name__ == "__main__":
    # Same as used in paper of Grokfast
    parser = ArgumentParser(description="Train a model on MNIST without custom sampling")
    
    parser.add_argument("--label", type=str, default="")
    parser.add_argument("--seed", type=int, default=0)
    
    parser.add_argument("--train_points", type=int, default=1000)
    parser.add_argument("--optimization_steps", type=int, default=10000)
    parser.add_argument("--batch_size", type=int, default=200)
    parser.add_argument("--loss_function", type=str, default="MSE") #MSE or CrossEntropy
    parser.add_argument("--optimizer", type=str, default="AdamW")
    parser.add_argument("--weight_decay", type=float, default=2.0)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--initialization_scale", type=float, default=8.0)
    parser.add_argument("--download_directory", type=str, default=".")
    parser.add_argument("--depth", type=int, default=3)
    parser.add_argument("--width", type=int, default=200)
    parser.add_argument("--activation", type=str, default="ReLU")

    # Grokfast
    parser.add_argument("--filter", type=str, choices=["none", "ma", "ema", "fir"], default="none")
    parser.add_argument("--alpha", type=float, default=0.8)
    parser.add_argument("--lamb", type=float, default=0.1)

    # Samples ranking
    parser.add_argument("--ema_alpha_sampl_rank", type=float, default=0.9)

    # Boolean arguements need this due to bad behavior of parser.parse_args
    def boolean_string(s):
        if s not in {"False", "True"}:
            raise ValueError("Not a valid boolean string")
        return s == "True"

    # These are the hyperparameters related to our online sampling filtering algorithm
    parser.add_argument("--appl_sampl_filter", type=boolean_string, default=True)  # If False, perform regular training
    parser.add_argument("--sampling_distr_upd_freq", type=int, default=1)  # How often to update the sampling distribution
    parser.add_argument("--top_k", type=float, default=0.1)  # Fraction of samples to select more frequently
    parser.add_argument("--top_k_sampling_prob", type=float, default=0.9)  # Probability of selecting a sample from the top-k
    parser.add_argument("--high_freq_better", type=boolean_string, default=True)  # If True, samples with higher frequency gradient content are considered better for training

    # -----------------------------------------------------------------
    # Try different hyperparameter values for your grid search here
    # -----------------------------------------------------------------
    args = parser.parse_args(
        [
            "--appl_sampl_filter", "False", # booleans as non strings in order to work
            "--sampling_distr_upd_freq", "1", # the rest as strings for some reason
            "--top_k", "0.1",
            "--top_k_sampling_prob", "0.9",
            "--high_freq_better", "True",
        ]
    )
    # -----------------------------------------------------------------
    # -----------------------------------------------------------------

    # Create arg.label for the filename of the saved results
    if not args.appl_sampl_filter:
        args.label = f"filter{args.filter}_sampling_{args.appl_sampl_filter}_v2"
    else:
        args.label = f"high_freq_{args.high_freq_better}_top_k_{args.top_k}_top_k_prob_{args.top_k_sampling_prob}_upd_freq_{args.sampling_distr_upd_freq}"

    # Training with time recording

    # Start the timer
    start_time = time.time()

    # Call your training function
    main(args)

    # End the timer
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Convert to minutes and seconds (optional)
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training completed in {int(minutes)} minutes and {int(seconds)} seconds.")
    print(f"label:{args.label}")

In [ ]:
args.label="filternone_sampling_False_v2"
analyze_results(args.label)

In [ ]:
if __name__ == "__main__":
    # Same as used in paper of Grokfast
    parser = ArgumentParser(description="Train a model on MNIST without custom sampling")
    
    parser.add_argument("--label", type=str, default="")
    parser.add_argument("--seed", type=int, default=1)
    
    parser.add_argument("--train_points", type=int, default=1000)
    parser.add_argument("--optimization_steps", type=int, default=10000)
    parser.add_argument("--batch_size", type=int, default=200)
    parser.add_argument("--loss_function", type=str, default="MSE") #MSE or CrossEntropy
    parser.add_argument("--optimizer", type=str, default="AdamW")
    parser.add_argument("--weight_decay", type=float, default=2.0)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--initialization_scale", type=float, default=8.0)
    parser.add_argument("--download_directory", type=str, default=".")
    parser.add_argument("--depth", type=int, default=3)
    parser.add_argument("--width", type=int, default=200)
    parser.add_argument("--activation", type=str, default="ReLU")

    # Grokfast
    parser.add_argument("--filter", type=str, choices=["none", "ma", "ema", "fir"], default="none")
    parser.add_argument("--alpha", type=float, default=0.8)
    parser.add_argument("--lamb", type=float, default=0.1)

    # Samples ranking
    parser.add_argument("--ema_alpha_sampl_rank", type=float, default=0.9)

    # Boolean arguements need this due to bad behavior of parser.parse_args
    def boolean_string(s):
        if s not in {"False", "True"}:
            raise ValueError("Not a valid boolean string")
        return s == "True"

    # These are the hyperparameters related to our online sampling filtering algorithm
    parser.add_argument("--appl_sampl_filter", type=boolean_string, default=True)  # If False, perform regular training
    parser.add_argument("--sampling_distr_upd_freq", type=int, default=1)  # How often to update the sampling distribution
    parser.add_argument("--top_k", type=float, default=0.1)  # Fraction of samples to select more frequently
    parser.add_argument("--top_k_sampling_prob", type=float, default=0.9)  # Probability of selecting a sample from the top-k
    parser.add_argument("--high_freq_better", type=boolean_string, default=True)  # If True, samples with higher frequency gradient content are considered better for training

    # -----------------------------------------------------------------
    # Try different hyperparameter values for your grid search here
    # -----------------------------------------------------------------
    args = parser.parse_args(
        [
            "--appl_sampl_filter", "False", # booleans as non strings in order to work
            "--sampling_distr_upd_freq", "1", # the rest as strings for some reason
            "--top_k", "0.1",
            "--top_k_sampling_prob", "0.9",
            "--high_freq_better", "True",
        ]
    )
    # -----------------------------------------------------------------
    # -----------------------------------------------------------------

    # Create arg.label for the filename of the saved results
    if not args.appl_sampl_filter:
        args.label = f"filter{args.filter}_sampling_{args.appl_sampl_filter}"
    else:
        args.label = f"high_freq_{args.high_freq_better}_top_k_{args.top_k}_top_k_prob_{args.top_k_sampling_prob}_upd_freq_{args.sampling_distr_upd_freq}"

    # Training with time recording

    # Start the timer
    start_time = time.time()

    # Call your training function
    main(args)

    # End the timer
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time

    # Convert to minutes and seconds (optional)
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Training completed in {int(minutes)} minutes and {int(seconds)} seconds.")
    print(f"label:{args.label}")

In [ ]:

args.label="filternone_sampling_False"
analyze_results(args.label)


In [ ]:
import itertools

parser = ArgumentParser(description="Train a model on MNIST without custom sampling")

parser.add_argument("--label", type=str, default="")
parser.add_argument("--seed", type=int, default=1)

parser.add_argument("--train_points", type=int, default=1000)
parser.add_argument("--optimization_steps", type=int, default=10000)
parser.add_argument("--batch_size", type=int, default=200)
parser.add_argument("--loss_function", type=str, default="MSE") #MSE or CrossEntropy
parser.add_argument("--optimizer", type=str, default="AdamW")
parser.add_argument("--weight_decay", type=float, default=2.0)
parser.add_argument("--lr", type=float, default=1e-3)
parser.add_argument("--initialization_scale", type=float, default=8.0)
parser.add_argument("--download_directory", type=str, default=".")
parser.add_argument("--depth", type=int, default=3)
parser.add_argument("--width", type=int, default=200)
parser.add_argument("--activation", type=str, default="ReLU")

# Grokfast
parser.add_argument("--filter", type=str, choices=["none", "ma", "ema", "fir"], default="ema")
parser.add_argument("--alpha", type=float, default=0.8)
parser.add_argument("--lamb", type=float, default=1)

# Samples ranking
parser.add_argument("--ema_alpha_sampl_rank", type=float, default=0.9)

# Boolean arguements need this due to bad behavior of parser.parse_args
def boolean_string(s):
    if s not in {"False", "True"}:
        raise ValueError("Not a valid boolean string")
    return s == "True"

# These are the hyperparameters related to our online sampling filtering algorithm
parser.add_argument("--appl_sampl_filter", type=boolean_string, default=True)  # If False, perform regular training
parser.add_argument("--sampling_distr_upd_freq", type=int, default=1)  # How often to update the sampling distribution
parser.add_argument("--top_k", type=float, default=0.1)  # Fraction of samples to select more frequently
parser.add_argument("--top_k_sampling_prob", type=float, default=0.9)  # Probability of selecting a sample from the top-k
parser.add_argument("--high_freq_better", type=boolean_string, default=True)  # If True, samples with higher frequency gradient content are considered better for training



In [ ]:
# Define possible values for each parameter
param_grid = {
    "top_k": [0.2],  # Convert to float
    "top_k_sampling_prob": [0.6, 0.8],  # Convert to float
    "sampling_distr_upd_freq": [100],  # Convert to int
    "high_freq_better": [False, True]  # Boolean parameter
}

# Generate all combinations of parameters
param_combinations = list(itertools.product(*param_grid.values()))

# Run main in a loop for each combination
for param_values in param_combinations:
    # Extract parameter values
    top_k = param_values[0]
    top_k_sampling_prob = param_values[1]
    sampling_distr_upd_freq = param_values[2]
    high_freq_better = param_values[3]  # Boolean value
    
    # Ensure boolean values are correctly formatted as strings for argparse
    high_freq_better_str = "True" if high_freq_better else "False"

    # Create args dynamically
    args_list = [
        "--top_k", str(top_k),
        "--top_k_sampling_prob", str(top_k_sampling_prob),
        "--sampling_distr_upd_freq", str(sampling_distr_upd_freq),
        "--high_freq_better", high_freq_better_str,
        "--label", f"{top_k}_{top_k_sampling_prob}_{sampling_distr_upd_freq}_{high_freq_better}"
    ]

    # Debug print statement
    print(f"\nRunning with parameters: {args_list}")

    # Parse the arguments dynamically
    args = parser.parse_args(args_list)

    # Call main() with the updated args
    main(args)

In [ ]:
labels = [
    f"{param_values[0]}_{param_values[1]}_{param_values[2]}_{param_values[3]}"
    for param_values in param_combinations
]


# Compare multiple accuracy curves on the same plot
plot_all_experiments_together(labels, show_only_val=True)